## Get tweets!

In [1]:
from pymongo import MongoClient, InsertOne, DeleteOne, ReplaceOne
import requests
from requests_oauthlib import OAuth1
import cnfg
from pprint import pprint
import json
import tweepy

In [2]:
client = MongoClient()
db = client["tweets"]
#db.create_collection("tweet_test")
tweets_collection = db.get_collection('tweet_test')


In [9]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [ ]:
class listener(StreamListener):
    def on_data(self, data):
        data = json.loads(data)
        tweets_collection.insert_one(data)

#        # Pull the fields we want, and throw it into our mongodb database
#        if data['lang'] == 'en':
#            tweet_document = {}

#            tweet_document['created_at'] = data[‘created_at’]
#            tweet_document[‘favorite_count’] = data[‘favorite_count’]
#            tweet_document[‘retweet_count’] = data[‘retweet_count’]

#            #Get full text if it is an extended tweet
#            if ‘extended_tweet’ in data.keys():
#                tweet_document[‘text’] = data[‘extended_tweet’][‘full_text’]
#            else:
#                tweet_document[‘text’] = data[‘text’]

#            tweet_document[‘screen_name’] = data[‘user’][‘screen_name’]

#            text = TextBlob(tweet_document[‘text’]).sentiment

#            tweet_document[‘sentiment’] = text.polarity
#            tweet_document[‘subjectivity’] = text.subjectivity

#           # print(tweet_document)
#           # print(‘\n’)
#            tweet_collection.insert_one(tweet_document)
        return True
    def on_error(self, status):
        print(status)

In [ ]:
twitterStream = Stream(auth, listener())
data = twitterStream.filter(track=privacy_search)

In [42]:
#check api limits --- probably not necessary with streaming data
import requests
from application_only_auth import Client
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
client = Client(consumer_key, consumer_secret)
response = client.request(
    "https://api.twitter.com/1.1/application/rate_limit_status.json?resources=help,users,search,statuses")
response['resources']['search']

## Old code

In [ ]:
# old search I tried on non-streaming API, use with caution or you may get your api key banned
privacy_search = 'facebook OR (data AND privacy) OR (data AND security) OR (data AND leak) OR (privacy AND violation)'

results=[]

for tweet in tweepy.Cursor(api.search,q=privacy_search).items(200):
    results.append(tweet)
    
for tweet in results:
    print (tweet._json['id'])
    if (tweets_collection.find_one({'id':tweet._json['id']})): 
        print ('result already in db')
    else:
        print('inserting new result')
        tweets_collection.insert_one(tweet._json)


In [58]:
#figuring things out code
tweets_collection.count()
len(list(tweets_collection.find({'id':1098020699694870529})))
if (tweets_collection.find_one({'id':1013755030401026})):
    print (True)
else:
    print (False)

1

In [90]:
#len(list(tweets_collection.find({},{'id'}))) #checks length of db in a dumb way
len(tweets_collection.distinct( "id" )) #checks for unique items

51895

In [91]:
tweets_collection.count() #there are a few dupes

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


52959